In [48]:
from Bio import SeqIO
import pandas as pd
import re

In [49]:
# import data
palmscan_hits = 'analysis_results/palmscan_hits.tsv'
df = pd.read_csv(palmscan_hits, sep='\t')
contigs_all = '/home/tobamo/analize/project-tobamo/analysis/analysis_data/test_contigs_2024-04-09.fasta'
contigs_palmscan_pos = '/home/tobamo/analize/project-tobamo/analysis/analysis_data/test_contigs_2024-04-09_palmscan_pos.fasta'
snakemake_results = pd.read_csv('/home/tobamo/analize/project-tobamo/results/megan6_results_combined.csv')
samples_path = '/home/tobamo/analize/project-tobamo/config/samples_test.tsv'
with open(samples_path) as file: samples = [line.strip() for line in file.readlines()]

In [50]:
# define function for removing last num from contig name
def remove_last_number(s):
    return re.sub(r"_(\d+)$", "", s)

# make new cols
df['contig'] = df['Label'].apply(lambda x: remove_last_number(x))
df["SRR"] = df["contig"].str.extract(r"_([A-Za-z0-9]+)$")

In [58]:
# define lists
contig_names = list(df.contig.unique())
SRRs = list(df.SRR.unique())
snakemake_filtered = snakemake_results[snakemake_results['SRR'].isin(samples)]

In [59]:
# print report
print(f'palmscan positive\nSRRs: {len(SRRs)}/{snakemake_filtered.SRR.nunique()}\ncontigs: {len(contig_names)}/{snakemake_filtered.qseqid.nunique()}')

palmscan positive
SRRs: 98/145
contigs: 221/2567


In [60]:
# make new filtered fasta
with open(contigs_all, "r") as file, open(contigs_palmscan_pos, "w") as outfile:
    out_seq = (r for r in SeqIO.parse(file, "fasta") if r.id in contig_names)
    SeqIO.write(out_seq, outfile, "fasta")